In [ ]:
!pip install --upgrade pytorch
!pip install pytorch-lightning==0.8.1
!pip uninstall torchtext
!pip install torchtext==0.6.0

In [ ]:
import torch


In [ ]:
!pip install sacrebleu

In [ ]:
from pytorch_lightning.core.lightning import LightningModule

In [ ]:
#clone haryoa recibrew transformer model
import sys
!git clone https://github.com/haryoa/ingredbrew
%load_ext autoreload
%autoreload 2
sys.path.append("ingredbrew")
%cd ingredbrew
!git reset --hard d91fe93ee26c462b9e480453868883fc1e83e4e8 #resets to correct version

In [ ]:
#import pytorch lightning
from recibrew.core_nn import TransformersLightning
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

seed_everything(888)

In [ ]:
#import nltk tokenizer
import nltk
nltk.download('punkt')

In [ ]:
%cd ..

In [ ]:
#instantiate transformer model
tl = TransformersLightning(train_csv='./data/processed/train_100k.csv', dev_csv='./data/processed/dev_10k.csv',
                           test_csv='./data/processed/eval_10k.csv', dim_feedforward=100, num_embedding=1000, lr=0.0001, dropout=0.1)

# Training

In [ ]:
#Only run if you want to train the model from a previous checkpoint
tl = tl.load_from_checkpoint('path/to/checkpoint')

In [ ]:
tl = tl.cuda()
#Display model training parameters
tl.hparams

In [ ]:
#instantiate trainer
import os

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=10,
   verbose=False,
   mode='auto'
)

tmpdir = '../models/'
ckpt = ModelCheckpoint(os.path.join(tmpdir, 'Model-{epoch}-{val_loss:.2f}'), save_top_k=2)
#Note: Trainer only saves the last two checkpoints
trainer = Trainer(early_stop_callback=early_stop_callback, checkpoint_callback=ckpt, 
                  accumulate_grad_batches=2,  auto_lr_find=False, gpus=[0]) 

In [ ]:
#View summary of model size
tl.summarize()

In [ ]:
#Train the model
trainer.fit(tl)

In [ ]:
#Run this if you want to manually save current checkpoint
trainer.save_checkpoint("final_checkpoint.ckpt")